# 1. Importing modules

In [1]:
import sys
import os
import numpy as np
import h5py

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Input, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import *
from tensorflow.keras.losses import *

# 2. Defining model
![alt text](figures/vgg16-class.png)

In [3]:
def getVGGModel():
    input_tensor = Input(shape=(224,224,3))
    vgg_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
    x = vgg_model.get_layer('block5_pool').output

    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(3, activation='softmax')(x)
    cus_model = keras.Model(inputs=vgg_model.input, outputs=x)

    return cus_model

In [4]:
model = getVGGModel()

# 3. Disabling trained VGG

In [5]:
for layer in model.layers[:19]:
    layer.trainable = False

# 4. Initializing optimizer

In [6]:
opt = keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999)

# 5. Data Augmentation

In [7]:
datagen = ImageDataGenerator(
#    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_it = datagen.flow_from_directory('/tmp/Dataset_2/Train/', target_size=(224,224), class_mode='categorical', batch_size=16, shuffle=True)
val_it = datagen.flow_from_directory('/tmp/Dataset_2/Validation/', target_size=(224,224), class_mode='categorical', batch_size=1, shuffle=False)

Found 1322 images belonging to 3 classes.
Found 363 images belonging to 3 classes.


# 6. Defining a customized loss function to smooth labels

### To deal with mis-labeling of data
### $new\_onehot\_labels = onehot\_labels * (1 - label\_smoothing) + label\_smoothing / num\_classes$

### Assuming label_smoothing = 0.2
### 0 — not damaged, 1 — most damaged

### A most damaged image would have lable [0, 1]
### $new\_onehot\_labels = [0, 1] * (1 - 0.2) + 0.2 / 2 =[0, 1]*(0.8) + 0.1$
### $new\_onehot\_labels =[0.1, 0.9]$

In [8]:
def cce_function(y_true, y_pred):
    return keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=0.1)

In [9]:
model.compile(loss=cce_function,
              optimizer=opt,
              metrics=['accuracy'])

# 7. Setting up checkpoints

# 8. Setting up learning rate reducer

In [11]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,
                              patience=5, min_lr=1e-8, verbose=1)

# 9. Training with label smoothing, checkpointing, and LR reducer
### Expect to see checkpoint message
```
val_accuracy improved from 0.49311 to 0.52066, saving model to model-02-0.52.hdf5
```

### Expect to see LR reducer message
```
Epoch 00028: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07
```

### sample output:
```
Epoch 1/50
83/83 [==============================] - 341s 4s/step - loss: 7.7586 - accuracy: 0.3964 - val_loss: 3.0279 - val_accuracy: 0.4931

Epoch 00001: val_accuracy improved from -inf to 0.49311, saving model to model-01-0.49.hdf5
Epoch 2/50
83/83 [==============================] - 298s 4s/step - loss: 6.0260 - accuracy: 0.5174 - val_loss: 0.4959 - val_accuracy: 0.5207

Epoch 00002: val_accuracy improved from 0.49311 to 0.52066, saving model to model-02-0.52.hdf5
Epoch 3/50
83/83 [==============================] - 306s 4s/step - loss: 5.7511 - accuracy: 0.5522 - val_loss: 1.5318 - val_accuracy: 0.5702

Epoch 00003: val_accuracy improved from 0.52066 to 0.57025, saving model to model-03-0.57.hdf5
Epoch 4/50
83/83 [==============================] - 302s 4s/step - loss: 5.6942 - accuracy: 0.5673 - val_loss: 1.0745 - val_accuracy: 0.6033

Epoch 00004: val_accuracy improved from 0.57025 to 0.60331, saving model to model-04-0.60.hdf5
Epoch 5/50
83/83 [==============================] - 307s 4s/step - loss: 5.6281 - accuracy: 0.5802 - val_loss: 1.0745 - val_accuracy: 0.6171

Epoch 00005: val_accuracy improved from 0.60331 to 0.61708, saving model to model-05-0.62.hdf5
Epoch 6/50
83/83 [==============================] - 306s 4s/step - loss: 5.0847 - accuracy: 0.6415 - val_loss: 1.0745 - val_accuracy: 0.5923
```

In [12]:
model.fit_generator(train_it,
                    steps_per_epoch=83,
                    callbacks = [checkpoint, reduce_lr],
                    validation_data=val_it,
                    validation_steps=363,
                    epochs=50)

/scratch1/00946/zzhang/python-envs/py3.8-torch1.10/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
32/83 [==========>...................] - ETA: 1:35 - loss: 13.5594 - accuracy: 0.3231

KeyboardInterrupt: 